In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../input/netflix-shows/netflix_titles.csv")

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df[df["title"] == "Breaking Bad"]

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
import missingno

missingno.bar(df)

In [ ]:
df.director.fillna("missing", inplace=True)
df.cast.fillna("missing", inplace=True)
df.country.fillna("missing", inplace=True)

df.isna().sum()

In [ ]:
df.dropna(subset=["date_added"], inplace=True)
df.dropna(subset=["rating"], inplace=True)

# df = df.dropna()

df.isna().sum()

In [ ]:
df.shape

In [ ]:
df = df.rename(columns={"listed_in": "genre"})
df["genre"] = df["genre"].apply(lambda x: x.split(", ")[0])

df.tail()

In [ ]:
df.country.value_counts()

Wir nehmen immer nur das erst genannte Land:

In [ ]:
df["country_main"] = df["country"].apply(lambda x: x.split(", ")[0])

df.tail()

In [ ]:
df["month_add"] = df["date_added"].apply(lambda x: x.split(" ")[0])
df["year_add"] = df["date_added"].apply(lambda x: x.split(" ")[-1])

df.head()

# Visualisierungen

In [ ]:
sns.countplot(x="type", data=df)

1. Welche Länder machen Filme/Serien
1. Welche Genres sind "beliebt"
1. Häufigkeit von Schauspielern
1. Welcher Direktor hat di meisten Filme produziert
1. Schauspieler x Genres
1. Filmdauer/Seriendauer (Durchschnitt & Summe)
1. Welches Jahr hat die meisten Filme/Serien
1. Rating (Für wen wede Filme produziert)

## Jahre

In [ ]:
temp_df = df[df["release_year"] > 2000]
sns.countplot(y="release_year", data=temp_df)

In [ ]:
plt.figure(figsize=(12,7))
sns.countplot(y="release_year", data=temp_df, hue="type")
plt.gca().invert_yaxis()

--- TBD: year_added

## Genres

In [ ]:
df.genre.value_counts()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x=df.genre.value_counts()[:10].sort_values().index, 
            y=df.genre.value_counts()[:10].sort_values().values)
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12,7))
sns.countplot(x="genre", data=df, order=df["genre"].value_counts()[:10].index)

## Country

In [ ]:
plt.figure(figsize=(12,7))
sns.countplot(x="country_main", data=df, order=df["country_main"].value_counts()[:10].index, hue="type")

## Staffeln

In [ ]:
tv_df = df[df["type"]=="TV Show"]

tv_df["duration"]=tv_df["duration"].str.replace(" Season", "")
tv_df["duration"]=tv_df["duration"].str.replace("s", "")
tv_df["duration"]=tv_df["duration"].astype(str).astype(int)
tv_df["duration"]

In [ ]:
tv_df.loc[:, ["title", "duration"]].sort_values(by="duration", ascending=False)[:5]

In [ ]:
tv_df["duration"].hist()

In [ ]:
tv_shows = df[df["type"]=="TV Show"]
tv_shows["duration"] = tv_shows["duration"].str.replace(" Season", "")
tv_shows["duration"] = tv_shows["duration"].str.replace("s", "")
tv_shows["duration"] = tv_shows["duration"].astype(str).astype(int)
tv_shows["duration"].head()

In [ ]:
tv_shows.sort_values(by="duration", ascending=False)[:1]

In [ ]:
tv_shows["duration"].hist()

In [ ]:
movies = df[df["type"]=="Movie"]
movies["duration"] = movies["duration"].str.replace(" min", "")
movies["duration"] = movies["duration"].astype(str).astype(int)
movies["duration"].head()

In [ ]:
sns.kdeplot(data=movies["duration"])

In [ ]:
duration_year = movies.groupby(["release_year"]).mean()
duration_year

plt.figure(figsize=(15,6))
sns.lineplot(x=duration_year.index, y=duration_year.duration.values)

In [ ]:
# 1 - get top ten countries
top_n_countries = movies["country_main"].value_counts()[:5].index.values
# ["US", "India", "UK", ...]

# 2 Filter movies
# datensatz[datensatz["Spalte"] OPERATOR Wert]
movies_new = movies[movies["country_main"].isin(top_n_countries)]
# movies_new.country_main.unique()

# 3 Groupby release_year und country_main -> Durchschnittliche Dauer pro Jahr für jedes Land
duration_year_country = movies_new.groupby(["release_year", "country_main"]).mean()
duration_year_country = duration_year_country.reset_index()

# 4 Plot
plt.figure(figsize=(15,6))
sns.lineplot(x=duration_year_country["release_year"], 
             y=duration_year_country["duration"], 
             hue=duration_year_country["country_main"])

# Recommender

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

train = ["The sky is blue.", "The sky is bright."]

countVectorizer = CountVectorizer(analyzer="word", stop_words="english")
tfidfVectorizer = TfidfVectorizer(analyzer="word", stop_words="english")

count_wm = countVectorizer.fit_transform(train)
tfidf_wm = tfidfVectorizer.fit_transform(train)

count_tokens = countVectorizer.get_feature_names()
tfidf_tokens = tfidfVectorizer.get_feature_names()

df_count = pd.DataFrame(data=count_wm.toarray(), index=["Doc1", "Doc2"], columns=count_tokens)
df_count

In [ ]:
df_tfidf = pd.DataFrame(data=tfidf_wm.toarray(), index=["Doc1", "Doc2"], columns=tfidf_tokens)
df_tfidf

In [ ]:
df["text"] = df["description"] + " " + df["genre"] + " " + df["type"]

In [ ]:
df["description"]

In [ ]:
tfidf = TfidfVectorizer(stop_words="english")
#tfidf = CountVectorizer(stop_words="english")

In [ ]:
df["text"] = df["text"].fillna("")

tfidf_matrix = tfidf.fit_transform(df["text"])

tfidf_matrix.shape

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

sim_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(df.index, index=df["title"]).drop_duplicates()
indices

In [ ]:
def get_recommendations(title, sim_matrix = sim_matrix):
    idx = indices[title]
    
    sim_scores = list(enumerate(sim_matrix[idx]))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # sim_scores.sort(_values)()
    
    sim_scores = sim_scores[1:100]
    
    movie_idx = [i[0] for i in sim_scores]
    
    return df["title"].iloc[movie_idx]

In [ ]:
get_recommendations("Breaking Bad")

In [ ]:
df[df["title"]=="Gothika"]